In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

/bin/bash: ls!apt-get: command not found
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.co

In [0]:
!ls

adc.json  drive  sample_data


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [1]:
cd drive/Houston

/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
area_lidar = []
for i in range(14):
    img_path = "area/lidar_area_" + str(i) + ".png"
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    area_lidar.append(img)

area_lidar = np.array(area_lidar)

In [4]:
print(area_lidar.shape)

(14, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
area_lidar_train_data = []
area_lidar_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            area_lidar_train_data.append(area_lidar[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            area_lidar_test_data.append(area_lidar[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
area_lidar_train_data = np.array(area_lidar_train_data)
area_lidar_test_data = np.array(area_lidar_test_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [9]:
print(area_lidar_train_data.shape)
print(area_lidar_test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 14)
(12197, 14)
(2832,)
(12197,)


In [10]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

Using TensorFlow backend.


In [11]:
print(train_one_hot.shape)
print(test_one_hot.shape)

(2832, 15)
(12197, 15)


In [0]:
LiDAR_PATCH_SIZE = 41
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005
BATCH_SIZE = 25

In [0]:
import numpy as np
padded_area_lidar = np.lib.pad(area_lidar, ((0,0), (LiDAR_PATCH_SIZE//2, LiDAR_PATCH_SIZE//2), (LiDAR_PATCH_SIZE//2,LiDAR_PATCH_SIZE//2)), 'reflect')

In [14]:
print(padded_area_lidar.shape)

(14, 389, 1945)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
area_lidar_train_patches = []
area_lidar_test_patches = []

for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      area_lidar_train_patches.append(get_patches(padded_area_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      area_lidar_test_patches.append(get_patches(padded_area_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))

In [0]:
area_lidar_train_patches = np.array(area_lidar_train_patches)
area_lidar_test_patches = np.array(area_lidar_test_patches)

In [18]:
print(area_lidar_train_patches.shape)
print(area_lidar_test_patches.shape)

(2832, 41, 41, 14)
(12197, 41, 41, 14)


In [19]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = area_lidar_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]


area_lidar_model = Sequential()
area_lidar_model.add(InputLayer(input_shape=(LiDAR_PATCH_SIZE, LiDAR_PATCH_SIZE, BANDS)))
area_lidar_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
area_lidar_model.add(BatchNormalization())
area_lidar_model.add(MaxPooling2D(pool_size=2, strides=2))
area_lidar_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
area_lidar_model.add(BatchNormalization())
area_lidar_model.add(MaxPooling2D(pool_size=2, strides=2))


area_lidar_model.add(Flatten())
area_lidar_model.add(Dense(FC1, activation='relu'))
area_lidar_model.add(Dropout(0.7))
area_lidar_model.add(Dense(FC2, activation='relu'))
area_lidar_model.add(Dropout(0.5))
area_lidar_model.add(Dense(NUM_CLS, activation='softmax'))

sgd = SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True)
area_lidar_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


area_lidar_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 21, 21, 500)       252500    
_________________________________________________________________
batch_normalization (BatchNo (None, 21, 21, 500)       2000      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 500)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 5, 5, 100)         1250100   
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 5, 100)         400       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 100)         0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
__________

In [42]:
history = area_lidar_model.fit(area_lidar_train_patches, train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=50)

Epoch 1/50
2832/2832 [==============================] - 3s 1ms/step - loss: 0.1251 - acc: 0.9555
Epoch 2/50
2832/2832 [==============================] - 3s 1ms/step - loss: 0.0902 - acc: 0.9633
Epoch 3/50
2832/2832 [==============================] - 3s 1ms/step - loss: 0.0853 - acc: 0.9689
Epoch 4/50
2832/2832 [==============================] - 3s 1ms/step - loss: 0.0971 - acc: 0.9661
Epoch 5/50
2832/2832 [==============================] - 3s 1ms/step - loss: 0.0856 - acc: 0.9725
Epoch 6/50
2832/2832 [==============================] - 3s 1ms/step - loss: 0.1390 - acc: 0.9580
Epoch 7/50
2832/2832 [==============================] - 3s 1ms/step - loss: 0.0931 - acc: 0.9675
Epoch 8/50
2832/2832 [==============================] - 3s 1ms/step - loss: 0.0868 - acc: 0.9696
Epoch 9/50
2832/2832 [==============================] - 3s 1ms/step - loss: 0.0779 - acc: 0.9696
Epoch 10/50
2832/2832 [==============================] - 3s 1ms/step - loss: 0.1014 - acc: 0.9682
Epoch 11/50
2832/2832 [======

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [43]:
test_cls = test_labels - 1
prediction = area_lidar_model.predict(area_lidar_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing



overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :\n", confusion_matrix)

Overall Accuracy of training sapmles :  0.5209477740427975
Average Accuracy of training samples :  0.5153237639783883
Kappa statistics of training samples :  0.4808995675869372
Each class accuracy of training samples :  [0.46482412 0.50488998 0.63937008 0.41722746 0.61830986 0.33944954
 0.43003064 0.4939577  0.38815117 0.8080402  0.75383228 0.46022727
 0.27272727 0.89671362 0.24210526]
Confusion matrix : [[370  93   0  89  29   2  89   2  96   0   3   2   7  14   0]
 [103 413   3  80   8   0  81   0  58   2   2  33  11   0  24]
 [  0   0 406   1   3   0   3   0   2   0   0   0   3   1 216]
 [ 15  32   0 310  33   0  56 117  65  17  13  43  31  10   1]
 [ 71  23   0  60 439   0   0   0  52   1  26   6   4  28   0]
 [ 46   0   0  53   2 111   0   4  71  14   2  24   0   0   0]
 [  1  87  29  72  12   6 421  48 117  24  51  15   4   0  92]
 [ 20 141   0 123 210   0  89 654   7  23  17  39   0   1   0]
 [ 69  39   0  84  14   0 114  16 380 101  87  19  20   0  36]
 [  1  47   0   2   0   0

In [0]:
area_lidar_model.save_weights('Models/area_lidar_model_weights.h5')